In [1]:
import os
import ast
import json
import glob
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
#import cudf
import joblib

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn import preprocessing

In [2]:
train_w1 = dict()
label_w1 = dict()
# train_w2 = dict()
# train_w3 = dict()
# train_w4 = dict()

for file in glob.glob("/kaggle/input/otto-prep-4-weeks/train_w0_part*.parquet"):
   train_w1[file] = pd.read_parquet(file)
train_w1 = pd.concat(train_w1.values())

for file in glob.glob("/kaggle/input/otto-prep-4-weeks/label_w0_part*.parquet"):
   label_w1[file] = pd.read_parquet(file)
label_w1 = pd.concat(label_w1.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w1_part*.parquet"):
#    train_w2[file] = pd.read_parquet(file)
# train_w2 = pd.concat(train_w2.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w2_part*.parquet"):
#    train_w3[file] = pd.read_parquet(file)
# train_w3 = pd.concat(train_w3.values())

# for file in glob.glob("/kaggle/input/otto-training-wo-split/train_w3_part*.parquet"):
#    train_w4[file] = pd.read_parquet(file)
# train_w4 = pd.concat(train_w4.values())


#max aid unique 486

In [3]:
batch_size = 20
history_size = 486


In [4]:
#label encoding aids

temp_all_data = pd.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/train.parquet')


le = preprocessing.LabelEncoder()
le.fit(temp_all_data['aid'])
del temp_all_data

train_w1['aid'] = le.transform(train_w1['aid'])

#set start = 2 to reserve 0 for mask and 1 for pad
train_w1['aid+2'] = train_w1['aid']+2
train_w1['aid'] = train_w1['aid+2']

train_w1 = train_w1.drop(['aid+2'], axis = 1)


joblib.dump(le, 'label_encoder.joblib')

['label_encoder.joblib']

In [12]:
train_w1 = pd.DataFrame(train_w1.groupby('session')['aid'].unique().agg(list))

In [17]:
label_w1 = pd.DataFrame(label_w1.groupby('session')['aid'].unique().agg(list))

In [21]:
label_w1.rename(columns = {'aid': 'target'}, inplace = True)
label_w1

,target
session,
0,"[1816325, 984597, 1072782, 173702, 1407538]"
1,"[1492293, 910862, 1491172, 424964, 1515526, 44..."
2,"[137492, 504789, 795863, 378348, 26638, 817441]"
3,"[357461, 984459, 622368, 578649, 1253857, 1660..."
4,[298827]
...,...
5348699,[1211923]
5348703,[1764745]
5348707,[1453498]


In [22]:
train_w1 = train_w1.reset_index()
label_w1 = label_w1.reset_index()

train_w1 = pd.merge(train_w1, label_w1, on = "session", how = "inner")

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, history_size=120):
        self.data = df
        self.history_size = history_size

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        

In [ ]:
train_w2 =add_day(train_w2)
train_w3 =add_day(train_w3)
train_w4 =add_day(train_w4)